In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
!pip install timm

In [14]:
import torch
from torch import nn, optim
from torch.utils.data import (Dataset, 
                              DataLoader,
                              TensorDataset)
import tqdm
import glob
import os.path as osp
import random
import numpy as np
import json
from PIL import Image
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import torch.utils.data as data
import torchvision
from torchvision import models, transforms

import timm
from sklearn.model_selection import StratifiedKFold


In [15]:
# 乱数のシードを設定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [ ]:
#import os
#path="/content/drive/MyDrive/signate_practice_chuzo/"
#zip_files=os.listdir(path)
#zip_files


['make_folders_and_data_downloads.ipynb',
 '1-5_fine_tuning.ipynb',
 '1_3_transfer_learning_on_GoogleColab.ipynb',
 'seti-learned-image-resizing.ipynb',
 'train_data.zip',
 'test_data.zip',
 'sample_submission.csv',
 'train.csv',
 'eca-nfnet-sam-opt-mixup-k-folds-w-b.ipynb',
 'pytorch-lightning-grid-mask-ranger-opt-w-b.ipynb',
 '__MACOSX',
 'signate_practice_chuzo.ipynb',
 'train',
 '.ipynb_checkpoints',
 'test',
 'valid',
 'input',
 'sub_vit_tpu_chuzo.csv',
 'ViT_SIGNATE_CHUZO_RAM30_tpu.ipynb',
 'PyTorch_Chapter4.ipynb',
 'signate_practice_chuzo2.ipynb']

In [ ]:
#import zipfile
#for i in zip_files:
#    if ".zip" in i:
#        with zipfile.ZipFile(path+'/'+i) as existing_zip:
#            existing_zip.extractall(path)
#    else:
#        continue

リスト4.5　DataLoaderを作成（<your_path>を変更している）

In [ ]:

#!cp -r /content/drive/MyDrive/signate_practice_chuzo/train_data /content/drive/MyDrive/signate_practice_chuzo/valid_data

In [ ]:
# 入力画像の前処理をするクラス
# 訓練時と推論時で処理が異なる


class ImageTransform():
    """
    画像の前処理クラス。訓練時、検証時で異なる動作をする。
    画像のサイズをリサイズし、色を標準化する。
    訓練時はRandomResizedCropとRandomHorizontalFlipでデータオーギュメンテーションする。


    Attributes
    ----------
    resize : int
        リサイズ先の画像の大きさ。
    mean : (R, G, B)
        各色チャネルの平均値。
    std : (R, G, B)
        各色チャネルの標準偏差。
    """

    def __init__(self, resize, mean, std):
        self.data_transform = {
            'train': transforms.Compose([
                transforms.Resize(resize),  # リサイズ                         
                #transforms.RandomResizedCrop(resize, scale=(0.5, 1.0)),  # データオーギュメンテーション
                transforms.RandomHorizontalFlip(),  # データオーギュメンテーション
                transforms.RandomVerticalFlip(),    # データオーギュメンテーション
                transforms.ToTensor(),  # テンソルに変換
                transforms.Normalize(mean, std)  # 標準化
            ]),
            'valid': transforms.Compose([
                transforms.Resize(resize),  # リサイズ
                #transforms.CenterCrop(resize),  # 画像中央をresize×resizeで切り取り
                transforms.ToTensor(),  # テンソルに変換
                transforms.Normalize(mean, std)  # 標準化
            ])
        }

    def __call__(self, img, phase='train'):
        """
        Parameters
        ----------
        phase : 'train' or 'valid'
            前処理のモードを指定。
        """
        return self.data_transform[phase](img)


In [ ]:
def make_datapath_list(phase="train"):
    """
    データのパスを格納したリストを作成する。

    Parameters
    ----------
    phase : 'train' or 'valid'
        訓練データか検証データかを指定する

    Returns
    -------
    path_list : list
        データへのパスを格納したリスト
    """

    rootpath = "/content/drive/MyDrive/signate_practice_chuzo/"
    target_path = osp.join(rootpath+phase+'_data/*.jpeg')
    print(target_path)

    path_list = []  # ここに格納する

    # globを利用してサブディレクトリまでファイルパスを取得する
    for path in glob.glob(target_path):
        path_list.append(path)

    return path_list


# 実行
train_list = make_datapath_list(phase="train")
valid_list = make_datapath_list(phase="valid")

#train_list
#valid_list

/content/drive/MyDrive/signate_practice_chuzo/train_data/*.jpeg
/content/drive/MyDrive/signate_practice_chuzo/valid_data/*.jpeg


In [ ]:
# 3. 画像の前処理と処理済み画像の表示
size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

In [ ]:
class ChuzoDataset(data.Dataset):
    """
   画像のDatasetクラス。PyTorchのDatasetクラスを継承。

    Attributes
    ----------
    file_list : リスト
        画像のパスを格納したリスト
    transform : object
        前処理クラスのインスタンス
    phase : 'train' or 'valid'
        学習か訓練かを設定する。
    """

    def __init__(self, file_list, transform=None, phase='train'):
        self.file_list = file_list  # ファイルパスのリスト
        self.transform = transform  # 前処理クラスのインスタンス
        self.phase = phase  # train or valの指定

    def __len__(self):
        '''画像の枚数を返す'''
        return len(self.file_list)

    def __getitem__(self, index):
        '''
        前処理をした画像のTensor形式のデータとラベルを取得
        '''

        # index番目の画像をロード
        img_path = self.file_list[index]
        img = Image.open(img_path)  # [高さ][幅][色RGB]

        # 画像の前処理を実施
        img_transformed = self.transform(
            img, self.phase)  # torch.Size([3, 224, 224])

        # 画像のラベルをファイル名から抜き出す
        if self.phase == "train":
            label = img_path[62:63]
        elif self.phase == "valid":
            label = img_path[62:63]

        # ラベルを数値に変更する
        if label == "d": #def
            label = 0
        elif label == "o":#ok
            label = 1

        return img_transformed, label


# 実行
train_dataset = ChuzoDataset(
    file_list=train_list, transform=ImageTransform(size, mean, std), phase='train')

valid_dataset = ChuzoDataset(
    file_list=valid_list, transform=ImageTransform(size, mean, std), phase='valid')

# 動作確認
index = 0
print(train_dataset.__getitem__(index)[0].size())
print(train_dataset.__getitem__(index)[1])

torch.Size([3, 224, 224])
0


In [ ]:
# ミニバッチのサイズを指定
batch_size = 32

# DataLoaderを作成
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

valid_dataloader = torch.utils.data.DataLoader(
    valid_dataset, batch_size=batch_size, shuffle=False)

# 辞書型変数にまとめる
dataloaders_dict = {"train": train_dataloader, "valid": valid_dataloader}

# 動作確認
batch_iterator = iter(dataloaders_dict["valid"])  # イテレータに変換
inputs, labels = next(batch_iterator)  # 1番目の要素を取り出す
print(inputs.size())
print(labels)


torch.Size([32, 3, 224, 224])
tensor([0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        1, 1, 0, 0, 0, 1, 1, 0])


In [ ]:
params = {
    'seed': 42,
    'model': 'eca_nfnet_l0',
    'size' : 224,
    'inp_channels': 3,
    'device': 'cuda',
    'lr': 1e-4,
    'weight_decay': 1e-6,
    'batch_size': 32,
    'num_workers' : 4,
    'epochs': 3,
    'out_features': 2,
    'name': 'CosineAnnealingLR',
    'T_max': 10,
    'min_lr': 1e-6,
    'nfolds': 5,
}

In [ ]:
class EcaNFNet(nn.Module):
    def __init__(self, model_name=params['model'], out_features=params['out_features'],
                 inp_channels=params['inp_channels'], pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained,
                                       in_chans=inp_channels)
        n_features = self.model.head.fc.in_features
        self.model.head.fc = nn.Linear(n_features, out_features, bias=True)    
        
    
    def forward(self, x):
        x = self.model(x)
        return x
net = EcaNFNet()

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/ecanfnet_l0_ra2-e3e9ac50.pth" to /root/.cache/torch/hub/checkpoints/ecanfnet_l0_ra2-e3e9ac50.pth


In [ ]:
# 損失関数の設定
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(params=net.parameters(),  lr=1e-4,momentum=0.9)


In [ ]:
# モデルを学習させる関数を作成


def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # 初期設定
    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # epochのループ
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        # epochごとの訓練と検証のループ
        for phase in ['train', 'valid']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数

            # 未学習時の検証性能を確かめるため、epoch=0の訓練は省略
            if (epoch == 0) and (phase == 'train'):
                continue

            # データローダーからミニバッチを取り出すループ
            for inputs, labels in tqdm(dataloaders_dict[phase]):

                # GPUが使えるならGPUにデータを送る
                inputs = inputs.to(device)
                labels = labels.to(device)

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)  # 損失を計算
                    _, preds = torch.max(outputs, 1)  # ラベルを予測

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    # 結果の計算
                    epoch_loss += loss.item() * inputs.size(0)  # lossの合計を更新
                    # 正解数の合計を更新
                    epoch_corrects += torch.sum(preds == labels.data)

            # epochごとのlossと正解率を表示
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))



In [ ]:
# 学習・検証を実行する
net.load_state_dict(torch.load("/content/drive/MyDrive/signate_practice_chuzo/eca_nfnet.pth"))
num_epochs=10
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

使用デバイス： cuda:0
Epoch 1/10
-------------


100%|██████████| 8/8 [00:03<00:00,  2.16it/s]


valid Loss: 0.2618 Acc: 0.9840
Epoch 2/10
-------------


100%|██████████| 8/8 [00:08<00:00,  1.02s/it]


train Loss: 0.2549 Acc: 0.9800


...

valid Loss: 0.1429 Acc: 0.9960
Epoch 10/10
-------------


100%|██████████| 8/8 [00:08<00:00,  1.00s/it]


train Loss: 0.1384 Acc: 0.9960


100%|██████████| 8/8 [00:03<00:00,  2.23it/s]

valid Loss: 0.1308 Acc: 0.9960


In [ ]:
torch.save(net.state_dict(),"/content/drive/MyDrive/signate_practice_chuzo/eca_nfnet.pth")

In [ ]:
test=pd.read_csv('/content/drive/MyDrive/signate_practice_chuzo/input/sample_submission.csv', header=None)
test.head()
test_jpeg = test[0].to_list()
test_jpeg

['19871.jpeg',
 '29934.jpeg',
 '39826.jpeg',
 '49888.jpeg',
 '59918.jpeg',
 '69975.jpeg',
 '79963.jpeg',
 '89914.jpeg',
 '99851.jpeg',
 '109996.jpeg',
 '119892.jpeg',
 '129893.jpeg',
 '139981.jpeg',
 '149997.jpeg',
 '159915.jpeg',
 '169903.jpeg',
 '179915.jpeg',
 '189978.jpeg',
 '199935.jpeg',
 '209923.jpeg',
 '219889.jpeg',
 '229958.jpeg',
 '239962.jpeg',
 '249889.jpeg',
 '259831.jpeg',
 '269945.jpeg',
 '279912.jpeg',
 '289969.jpeg',
 '299953.jpeg',
 '309928.jpeg',
 '319945.jpeg',
 '329953.jpeg',
 '339986.jpeg',
 '349836.jpeg',
 '359973.jpeg',
 '369861.jpeg',
 '379924.jpeg',
 '389908.jpeg',
 '399924.jpeg',
 '409965.jpeg',
 '419909.jpeg',
 '429933.jpeg',
 '439948.jpeg',
 '449821.jpeg',
 '459909.jpeg',
 '469837.jpeg',
 '479840.jpeg',
 '489913.jpeg',
 '499929.jpeg',
 '509856.jpeg',
 '519895.jpeg',
 '529883.jpeg',
 '539843.jpeg',
 '549992.jpeg',
 '559880.jpeg',
 '569906.jpeg',
 '579910.jpeg',
 '589947.jpeg',
 '599930.jpeg',
 '609988.jpeg',
 '619967.jpeg',
 '629822.jpeg',
 '639930.jpeg',
 

In [ ]:
def make_datapath_list(phase="test"):
    """
    データのパスを格納したリストを作成する。

    Parameters
    ----------
    phase : 'train' or 'valid'
        訓練データか検証データかを指定する

    Returns
    -------
    path_list : list
        データへのパスを格納したリスト
    """
    test_path_list=[]   
    rootpath = "/content/drive/MyDrive/signate_practice_chuzo/"
    #test_path = osp.join(rootpath+phase+'_data/*.jpeg')
    for jpeg_path in test_jpeg:
      test_path = osp.join(rootpath+phase+'_data/'+jpeg_path)
      test_path_list.append(test_path)

    #path_list = []  # ここに格納する

    # globを利用してサブディレクトリまでファイルパスを取得する
    #for path in glob.glob(test_path):
    #    path_list.append(path)

    return test_path_list


# 実行
test_list  = make_datapath_list(phase="test")

test_list

['/content/drive/MyDrive/signate_practice_chuzo/test_data/19871.jpeg',
 '/content/drive/MyDrive/signate_practice_chuzo/test_data/29934.jpeg',
 '/content/drive/MyDrive/signate_practice_chuzo/test_data/39826.jpeg',
 '/content/drive/MyDrive/signate_practice_chuzo/test_data/49888.jpeg',
 '/content/drive/MyDrive/signate_practice_chuzo/test_data/59918.jpeg',
 '/content/drive/MyDrive/signate_practice_chuzo/test_data/69975.jpeg',
 '/content/drive/MyDrive/signate_practice_chuzo/test_data/79963.jpeg',
 '/content/drive/MyDrive/signate_practice_chuzo/test_data/89914.jpeg',
 '/content/drive/MyDrive/signate_practice_chuzo/test_data/99851.jpeg',
 '/content/drive/MyDrive/signate_practice_chuzo/test_data/109996.jpeg',
 '/content/drive/MyDrive/signate_practice_chuzo/test_data/119892.jpeg',
 '/content/drive/MyDrive/signate_practice_chuzo/test_data/129893.jpeg',
 '/content/drive/MyDrive/signate_practice_chuzo/test_data/139981.jpeg',
 '/content/drive/MyDrive/signate_practice_chuzo/test_data/149997.jpeg',
 

In [ ]:
class ChuzoTestDataset(data.Dataset):
    """
   画像のDatasetクラス。PyTorchのDatasetクラスを継承。

    Attributes
    ----------
    file_list : リスト
        画像のパスを格納したリスト
    transform : object
        前処理クラスのインスタンス
   """

    def __init__(self, file_list, transform=None):
        self.file_list = file_list  # ファイルパスのリスト
        self.transform = transform  # 前処理クラスのインスタンス
        #self.phase = phase  # train or valの指定

    def __len__(self):
        '''画像の枚数を返す'''
        return len(self.file_list)

    def __getitem__(self, index):
        '''
        前処理をした画像のTensor形式のデータとラベルを取得
        '''

        # index番目の画像をロード
        img_path = self.file_list[index]
        img = Image.open(img_path)  # [高さ][幅][色RGB]

        # 画像の前処理を実施
        img_transformed = self.transform(
            img)  # torch.Size([3, 224, 224])


        return img_transformed


# 実行

test_dataset = ChuzoTestDataset(
    file_list=test_list, transform=ImageTransform(size, mean, std))

# 動作確認
index = 0
print(test_dataset.__getitem__(index)[0].size())

torch.Size([224, 224])


In [ ]:
test_loader = DataLoader(
    test_dataset, batch_size=10, shuffle=False)

In [ ]:
# 動作確認
test_iterator = iter(test_loader)  # イテレータに変換
inputs = next(test_iterator)  # 1番目の要素を取り出す
print(inputs.size())


torch.Size([10, 3, 224, 224])


In [ ]:
import numpy as np
net.to("cuda:0")
net.eval()

ypreds = []
for x in test_loader:
        # toメソッドで計算を実行するデバイスに転送する
    x = x.to("cuda:0")
        #y = y.to(device)
        # 確率が最大のクラスを予測(リスト2.14参照)
        # ここではforward（推論）の計算だけなので自動微分に
        # 必要な処理はoffにして余計な計算を省く
    with torch.no_grad():
      _, y_pred = net(x).max(1)
    #ys.append(y)
    #y_pred = y_pred.to("cpu")
    ypreds.append(y_pred)
    # ミニバッチごとの予測結果などを1つにまとめる
   # ys = torch.cat(ys)
#ypreds = np.vstack(ypreds)
ypreds = torch.cat(ypreds)
    # 予測精度を計算
   

In [ ]:
ypreds = ypreds.to("cpu").numpy()
ypreds

array([1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0])

In [ ]:
test=pd.read_csv('/content/drive/MyDrive/signate_practice_chuzo/input/sample_submission.csv', header=None)
test.head()

,0,1
0,19871.jpeg,0
1,29934.jpeg,0
2,39826.jpeg,0
3,49888.jpeg,0
4,59918.jpeg,0


In [ ]:
import pandas as pd
pred_df=pd.DataFrame(data=ypreds)
pred_df.head()

,0
0,1
1,1
2,1
3,0
4,1


In [ ]:
test[1]=pred_df
test

,0,1
0,19871.jpeg,1
1,29934.jpeg,1
2,39826.jpeg,1
3,49888.jpeg,0
4,59918.jpeg,1
...,...,...
95,969983.jpeg,0
96,979891.jpeg,0
97,989921.jpeg,1
98,999921.jpeg,0


In [ ]:
test.to_csv('/content/drive/MyDrive/signate_practice_chuzo/sub_pytorch_chuzo3.csv', header=None, index=False)